# Single-particle Eigenstate Thermolization in Polariton Hamiltonians. Photonic Wire Model
**Ribeiro Group Rotation Project A**

Brian Zhao, 09 Sept 2022

The following is adapted from Ribeiro, R. F. _Multimode Polariton Effects on Molecular Energy Transport and Spectral Fluctuations_. Commun Chem 2022, 5 (1), 1–10. https://doi.org/10.1038/s42004-022-00660-0.

### Model Hamiltonian
#### Cavity Hamiltonian
The empty cavity Hamiltonian is given by
$$
H_{\text{L}}=\sum_q\hbar\omega_q a_q^{\dagger}a_q,
$$
where
$$
\omega_q = \frac{c}{\sqrt{\epsilon}}\sqrt{q_0^2+q^2},
$$
where $q_0=\sqrt{(\pi/L_z)^2+(\pi/L_y)^2}$ is a constant, and $q=2\pi m/L$ ($m\in \mathcal{Z}$) are the _cavity modes_.

#### Matter Hamiltonian
The Hamiltonian for the molecules are given by
$$
H_{\text{M}}=\sum_{i=1}^{N_{\text{M}}}(\hbar\omega_{\text{M}}+\sigma_i)b_i^+b_i^-,
$$
where $b_i^+=|1_i\rangle\langle 0_i|$ and $b_i^-=|0_i\rangle\langle 1_i|$ creates and annihilates an excitation at the $i$-th molecule respectively, and $\sigma_i$ is drawn from a normal distribution with variance $\sigma^2$.

#### Light-matter Hamiltonian
Applying the Coulomb gauge in the rotating-wave approximation (ignoring double raising and lowering), we have
$$
H_{\text{LM}}=\sum_{j=1}^{N_{\text{M}}}\sum_q\frac{-i\Omega_{\text{R}}}{2}\sqrt{\frac{\omega_{\text{M}}}{N_{\text{M}}\omega_q}}\frac{\mu_j}{\mu_0}\left(e^{iqx_j}b_j^+a_q-e^{iqx_j}a_q^{\dagger}b_j^- \right),
$$
where $\Omega_{\text{R}}=\mu_0\sqrt{\hbar\omega_0N_{\text{M}}/2\epsilon LL_yL_z}$, and $\mu_j$ is drawn from a normal distribution with variance $\sigma_{\mu}^2$.

We assume there is only one photon.

In [35]:
import numpy as np
import scipy.constants as cst

q_to_ev = cst.c*cst.hbar/cst.e

def e_q(m,L,Ly,Lz,eps):
    return np.sqrt((np.pi/Lz)**2 + (np.pi/Ly)**2 + (2*np.pi*m/L)**2)*q_to_ev/np.sqrt(eps)

def total_hamil_naive(num_mol, omega_r, a, Ly, Lz, eps, em_mean, em_sigma, mu_sigma, f):
    try:
        assert (num_mol-1)%2 == 0
    except AssertionError:
        raise ValueError('num_mol needs to be 2N+1, where N is a positive integer!')
        
    n = int((num_mol-1)/2)
    L = num_mol*a
    
    # Initialise our random number generator
    rng = np.random.default_rng()
    
    diag = np.concatenate((np.array([e_q(_,L,Ly,Lz,eps) for _ in range(-n,n+1)]),np.array([rng.normal(em_mean, em_sigma) for _ in range(num_mol)])))
    
    hamil = np.diag(diag) # Fill in the diagonal elements
    hamil = hamil.astype('complex128')
    
    # Fill in the off-diagonal elements
    # We just fill in the light-matter block (first quadrant), and take the complex conjugate for the third quadrant.
    for i in range(num_mol):
        # This is the photon loop, we can just compute mu_j once
        mu_j = rng.normal(1, mu_sigma)
        # omega_q was already computed in _diag
        omega_q = diag[i]
        for j in range(num_mol,2*num_mol):
            hamil[i,j] = (omega_r/2)*np.sqrt(diag[j]/(num_mol*omega_q))*(mu_j)*np.exp(((i*a*(j-num_mol)+rng.uniform(-f,f))-1.5*np.pi)*1j)
            hamil[j,i] = np.conjugate(hamil[i,j])
    
    return hamil

In [39]:
total_hamil_naive(3, 0.3, 10e-9, 400e-9, 200e-9, 3, 2.0, 0.2, 0.05, 0.1)

array([[ 2.39445087e+01+0.j        ,  0.00000000e+00+0.j        ,
         0.00000000e+00+0.j        ,  2.15242838e-03+0.02812124j,
        -1.18375952e-03+0.02674553j,  1.72296005e-04+0.02879491j],
       [ 0.00000000e+00+0.j        ,  2.00078640e+00+0.j        ,
         0.00000000e+00+0.j        , -5.53556370e-05+0.09643007j,
         6.66555321e-03+0.09129168j,  6.45090291e-03+0.09824235j],
       [ 0.00000000e+00+0.j        ,  0.00000000e+00+0.j        ,
         2.39445087e+01+0.j        , -1.80064759e-03+0.03099425j,
         2.89103977e-04+0.02946898j, -2.29381485e-03+0.031615j  ],
       [ 2.15242838e-03-0.02812124j, -5.53556370e-05-0.09643007j,
        -1.80064759e-03-0.03099425j,  2.39154451e+00+0.j        ,
         0.00000000e+00+0.j        ,  0.00000000e+00+0.j        ],
       [-1.18375952e-03-0.02674553j,  6.66555321e-03-0.09129168j,
         2.89103977e-04-0.02946898j,  0.00000000e+00+0.j        ,
         2.15488883e+00+0.j        ,  0.00000000e+00+0.j        ],
     